References :<br>
https://www.youtube.com/watch?v=6zm9NC9uRkk <br>
https://github.com/skipgram/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb

In [2]:
import os
data_directory = os.path.join('.','yelp_dataset')

businesses_filepath = os.path.join(data_directory, 'business.json')

with open(businesses_filepath) as f:
    first_business_record = f.readline() 
    
print(first_business_record)

{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}



In [3]:
#Reviews

review_path = os.path.join(data_directory,'review.json')

with open(review_path) as f:
    first_review = f.readline()
    
print(first_review)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [4]:
#Extract all business ids
import json

restaurant_ids=set()

with open(businesses_filepath) as f:
    
    for business_record in f:
        business = json.loads(business_record)
        
        if not business.get('categories'):
            continue
        
        if 'Restaurants' not in business['categories']:
            continue
            

        restaurant_ids.add(business['business_id'])
        
restaurant_ids = frozenset(restaurant_ids)

print(f'{len(restaurant_ids)} Restaurants in Dataset')

63944 Restaurants in Dataset


In [5]:
#Create a cratch directory to store all reviews
scratch_path = os.path.join('.','scratch')
try:
    os.mkdir(scratch_path)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_path,'review_text_all.txt')

In [6]:
#Extract all reviews snd store it in the new file created

execute = False

review_count = 0

if execute:
    
    with open(review_txt_filepath,"w") as review_txt_file:
        
        with open(review_path) as review_file:
            
            for review_json in review_file:
                review = json.loads(review_json)
                
                if review['business_id'] not in restaurant_ids:
                    continue
                    
                review_txt_file.write(review['text'].replace("\n","\\n")+"\n")
                review_count+=1
                
    print(f'Text from {review_count:,} restaurant reviews written to the new txt file.')

else:
    
    # count the reviews in the existing file
    with open(review_txt_filepath) as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Text from {review_count + 1:,} restaurant reviews in the txt file.')
                
                


Text from 5,058,162 restaurant reviews in the txt file.


In [7]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

nlp = spacy.load('en_core_web_sm')

In [8]:
#Length of each review
review_dict={}
with open(review_txt_filepath) as review_txt_file:
    review_dict = {review_count:len(line) for review_count,line in enumerate(review_txt_file)}
        
    

In [9]:
max_key = max(review_dict,key=review_dict.get)
print(max_key)

1788257


In [10]:
#Sample Review

review_num = max_key
with open(review_txt_filepath) as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

WED, MARCH 27, 2013 // 12:00PM-2:00PM 
                 PRICE FOR 2: $63.98 (TR price) + $5.18 (tax) + $10.84 (tip) = $80.00
                 WAIT: 15-20mins
                 SERVER: DAWN - (4/5). She looked frazzled but always made time to check up on us. Asked me if I wanted more Peachy Tea when I finished, refilled our waters, & brought us 2 carafes when we asked for coffee.

FRI, MARCH 29, 2013 // 5:00PM-7:30PM
                 PRICE FOR 2: $99.98 (TR price) + $8.10 (tax) + $16.42 (tip) = $124.50
                 WAIT: 25-30mins
                 SERVER: IRANDA - (1/5). We had to chase her down 3x, she ignored our table completely. When we asked for coffee, she brought us 1/2 empty cups (spilled coffee all over the saucers). VINCE (4/5), not our server, was friendly, paid closer attention & cleared our table multiple times.
_________________________

Bacchanal can be overwhelming since there are a LOT of dishes to choose from. Between Bacchanal & Wicked Spoon, if you only have time 

In [14]:
%%time
parsed_view = nlp(sample_review)

CPU times: user 327 ms, sys: 19.9 ms, total: 347 ms
Wall time: 141 ms


In [16]:
parsed_view = nlp(sample_review)

In [17]:
print(parsed_view)

WED, MARCH 27, 2013 // 12:00PM-2:00PM 
                 PRICE FOR 2: $63.98 (TR price) + $5.18 (tax) + $10.84 (tip) = $80.00
                 WAIT: 15-20mins
                 SERVER: DAWN - (4/5). She looked frazzled but always made time to check up on us. Asked me if I wanted more Peachy Tea when I finished, refilled our waters, & brought us 2 carafes when we asked for coffee.

FRI, MARCH 29, 2013 // 5:00PM-7:30PM
                 PRICE FOR 2: $99.98 (TR price) + $8.10 (tax) + $16.42 (tip) = $124.50
                 WAIT: 25-30mins
                 SERVER: IRANDA - (1/5). We had to chase her down 3x, she ignored our table completely. When we asked for coffee, she brought us 1/2 empty cups (spilled coffee all over the saucers). VINCE (4/5), not our server, was friendly, paid closer attention & cleared our table multiple times.
_________________________

Bacchanal can be overwhelming since there are a LOT of dishes to choose from. Between Bacchanal & Wicked Spoon, if you only have time 

In [18]:
for num,sentence in enumerate(parsed_view.sents):
    print(f'Sentence {num+1}')
    print(sentence)
    print('')

Sentence 1
WED, MARCH 27, 2013 //

Sentence 2
12:00PM-2:00PM 
                 PRICE FOR 2: $63.98 (TR price) + $5.18 (tax) + $10.84 (tip)

Sentence 3
=

Sentence 4
$80.00
                 

Sentence 5
WAIT: 15-20mins
                 SERVER: DAWN - (4/5).

Sentence 6
She looked frazzled but always made time to check up on us.

Sentence 7
Asked me if I wanted more Peachy Tea

Sentence 8
when I finished, refilled our waters, & brought us 2 carafes when we asked for coffee.



Sentence 9
FRI, MARCH 29, 2013 //

Sentence 10
5:00PM-7:30PM
                 PRICE FOR 2: $99.98 (TR price) + $8.10 (tax) + $16.42 (tip)

Sentence 11
=

Sentence 12
$124.50
                 

Sentence 13
WAIT:

Sentence 14
25-30mins
                 SERVER: IRANDA - (1/5).

Sentence 15
We had to chase her down 3x, she ignored our table completely.

Sentence 16
When we asked for coffee, she brought us 1/2 empty cups (spilled coffee all over the saucers).

Sentence 17
VINCE (4/5), not our server, was friendly, paid 

### Text Normalization

In [23]:
token_text = [token.orth_ for token in parsed_view]
token_lemma = [token.lemma_ for token in parsed_view]
token_shape = [token.shape_ for token in parsed_view]

pd.DataFrame(zip(token_text,token_lemma,token_shape),columns=["token_text","token_lemma","token_shape"])

,token_text,token_lemma,token_shape
0,WED,WED,XXX
1,",",",",","
2,MARCH,MARCH,XXXX
3,27,27,dd
4,",",",",","
5,2013,2013,dddd
6,//,//,//
7,12:00PM-2:00PM,12:00pm-2:00pm,dd:ddXX-d:ddXX
8,\n,\n,\n
9,PRICE,PRICE,XXXX


### Part of Speech

In [24]:
token_pos = [token.pos_ for token in parsed_view]
pd.DataFrame(zip(token_text,token_pos),columns=["Text","POS"])

,Text,POS
0,WED,PROPN
1,",",PUNCT
2,MARCH,PROPN
3,27,NUM
4,",",PUNCT
5,2013,NUM
6,//,SYM
7,12:00PM-2:00PM,NUM
8,\n,SPACE
9,PRICE,PROPN


In [25]:
displacy.render(parsed_view,style="ent")

In [27]:
for num, entity in enumerate(parsed_view.ents):
    print(f'Entity {num + 1}:', entity, '-', entity.label_)
    print('')

Entity 1: WED - ORG

Entity 2: MARCH - ORG

Entity 3: 12:00PM-2:00PM - CARDINAL

Entity 4: PRICE - ORG

Entity 5: 2 - CARDINAL

Entity 6: 63.98 - MONEY

Entity 7: 5.18 - MONEY

Entity 8: 10.84 - MONEY

Entity 9: 80.00 - MONEY

Entity 10: WAIT - ORG

Entity 11: 15 - CARDINAL

Entity 12: 4/5 - CARDINAL

Entity 13: Peachy Tea - ORG

Entity 14: FRI - ORG

Entity 15: MARCH - ORG

Entity 16: 2 - CARDINAL

Entity 17: 99.98 - MONEY

Entity 18: 8.10 - MONEY

Entity 19: 16.42 - MONEY

Entity 20: 124.50 - MONEY

Entity 21: WAIT - ORG

Entity 22: 25 - CARDINAL

Entity 23: 1/5 - CARDINAL

Entity 24: 3x - CARDINAL

Entity 25: 1/2 - CARDINAL

Entity 26: VINCE - ORG

Entity 27: 4/5 - CARDINAL

Entity 28: closer attention & - ORG

Entity 29: Bacchanal & Wicked Spoon - ORG

Entity 30: Bacchanal - ORG

Entity 31: CREPES - ORG

Entity 32: 1 - CARDINAL

Entity 33: 2 - CARDINAL

Entity 34: Rib - PERSON

Entity 35: 3 - CARDINAL

Entity 36: Crepes - ORG

Entity 37: 4 - DATE

Entity 38: 5 - CARDINAL

Entity 39

In [29]:
token_entity_type = [token.ent_type_ for token in parsed_view]
token_entity_iob = [token.ent_iob_ for token in parsed_view]

pd.DataFrame(
    zip(token_text, token_entity_type, token_entity_iob),
    columns=['token_text', 'entity_type', 'inside_outside_begin']
    )

,token_text,entity_type,inside_outside_begin
0,WED,ORG,B
1,",",,O
2,MARCH,ORG,B
3,27,,O
4,",",,O
5,2013,,O
6,//,,O
7,12:00PM-2:00PM,CARDINAL,B
8,\n,,O
9,PRICE,ORG,B


In [44]:
token_attributes = [
    (
        token.orth_,
        token.prob,
        token.is_stop,
        token.is_punct,
        token.is_space,
        token.like_num,
        token.is_oov,
        
    )for token in parsed_view
    
]

df = pd.DataFrame(token_attributes,columns = [
    'text',
    'log_probability',
    'stop?',
    'punctuation?',
    'space',
    'number',
    'out of vocabulary']
    )

df.loc[:,'stop?':'out of vocabulary'] = df.loc[:,'stop?':'out of vocabulary'].applymap(lambda x : x if x else "")
df

,text,log_probability,stop?,punctuation?,space,number,out of vocabulary
0,WED,-20.0,,,,,True
1,",",-20.0,,True,,,True
2,MARCH,-20.0,,,,,True
3,27,-20.0,,,,True,True
4,",",-20.0,,True,,,True
5,2013,-20.0,,,,True,True
6,//,-20.0,,True,,,True
7,12:00PM-2:00PM,-20.0,,,,,True
8,\n,-20.0,,,True,,True
9,PRICE,-20.0,,,,,True
